In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:

%pip install datasets --quiet

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import pandas as pd
from datasets import load_dataset
import os
import re

# Download the dataset
dataset = load_dataset("princeton-nlp/SWE-bench_Lite")

# Convert the dataset to a Pandas DataFrame
df = pd.DataFrame(dataset["dev"])

# Print the DataFrame
print(df.head())

                repo              instance_id  \
0  sqlfluff/sqlfluff  sqlfluff__sqlfluff-1625   
1  sqlfluff/sqlfluff  sqlfluff__sqlfluff-2419   
2  sqlfluff/sqlfluff  sqlfluff__sqlfluff-1733   
3  sqlfluff/sqlfluff  sqlfluff__sqlfluff-1517   
4  sqlfluff/sqlfluff  sqlfluff__sqlfluff-1763   

                                base_commit  \
0  14e1a23a3166b9a645a16de96f694c77a5d4abb7   
1  f1dba0e1dd764ae72d67c3d5e1471cf14d3db030   
2  a1579a16b1d8913d9d7c7d12add374a290bcc78c   
3  304a197829f98e7425a46d872ada73176137e5ae   
4  a10057635e5b2559293a676486f0b730981f037a   

                                               patch  \
0  diff --git a/src/sqlfluff/rules/L031.py b/src/...   
1  diff --git a/src/sqlfluff/rules/L060.py b/src/...   
2  diff --git a/src/sqlfluff/rules/L039.py b/src/...   
3  diff --git a/src/sqlfluff/core/parser/helpers....   
4  diff --git a/src/sqlfluff/core/linter/linted_f...   

                                          test_patch  \
0  diff --git a/test/cli/comm

In [ ]:
# Define a function to extract file names from git diff output
def extract_filenames(patch):
    pattern = r"diff --git a/(.*?) b/(.*?)\n"
    matches = re.findall(pattern, patch)
    # Return a list of unique file names
    return list(set([match[0] for match in matches]))


# Apply the function to the 'patch' column
df["edited_files"] = df["patch"].apply(extract_filenames)

In [ ]:
df_first_row = df.iloc[0:1]

In [ ]:
downloaded_repos = {}

In [ ]:
import uuid
from blar_graph.graph_construction.core.graph_builder import GraphConstructor
from blar_graph.db_managers import Neo4jManager
from blar_graph.agents_tools.agents_examples.metamate import get_metamate_agent
import subprocess
import hashlib


def generate_hash(data):
    # Convert data to bytes if it's not already
    if isinstance(data, str):
        data = data.encode("utf-8")

    # Generate the SHA-256 hash
    hashed_data = hashlib.sha256(data).hexdigest()

    return hashed_data


try:
    os.makedirs("repos", exist_ok=True)  # exist_ok=True avoids error if directory already exists
    print("Directory 'repos' ensured.")
except Exception as e:
    print(f"An error occurred ensuring the directory: {e}")

for index, row in df_first_row.iterrows():
    repo = row["repo"]
    print(repo)
    if downloaded_repos.get(repo) is None:
        repo_path = f"repos/{repo}"
        if not os.path.exists(f"repos/{repo}"):
            os.makedirs(repo_path, exist_ok=True)
            subprocess.run(f"git clone https://github.com/{repo}.git {repo_path}", shell=True, check=True)
            subprocess.run(f"git reset --hard {row['base_commit']}", shell=True, check=True)
            subprocess.run(f"git clean -fdxq", shell=True, check=True)
            subprocess.run(f"find {repo_path} -type l -delete", shell=True, check=True)
        repo_id = generate_hash(row["instance_id"])
        print(repo_id)
        graph_manager = Neo4jManager(repo_id)
        graph_constructor = GraphConstructor(graph_manager, "python")
        graph_constructor.build_graph(f"repos/{repo}")
        downloaded_repos[repo] = graph_manager

    graph_manager = downloaded_repos[repo]
    agent = get_metamate_agent(graph_manager)
    list(agent.stream({"input": f"Proble: {row['problem_statement']}\nHints: {row['hints_text']}"}))

Directory 'repos' ensured.
sqlfluff/sqlfluff
14661b1b5c8e93918d473fa837bb34631d66a5b414aebdc0667d1f69c299138a
Processed repos/sqlfluff/sqlfluff/examples/05_simple_api_config.py
Processed repos/sqlfluff/sqlfluff/examples/02_timing_api_steps.py
Processed repos/sqlfluff/sqlfluff/examples/01_basic_api_usage.py
Processed repos/sqlfluff/sqlfluff/examples/03_getting_rules_and_dialects.py
Processed repos/sqlfluff/sqlfluff/examples/04_config_overrides.py
Processed repos/sqlfluff/sqlfluff/plugins/sqlfluff-plugin-example/setup.py
Processed repos/sqlfluff/sqlfluff/plugins/sqlfluff-plugin-example/src/sqlfluff_plugin_example/rules.py
Processed repos/sqlfluff/sqlfluff/plugins/sqlfluff-plugin-example/src/sqlfluff_plugin_example/__init__.py
Processed repos/sqlfluff/sqlfluff/plugins/sqlfluff-templater-dbt/setup.py
Processed repos/sqlfluff/sqlfluff/plugins/sqlfluff-templater-dbt/sqlfluff_templater_dbt/__init__.py
Processed repos/sqlfluff/sqlfluff/plugins/sqlfluff-templater-dbt/sqlfluff_templater_dbt/temp

Failed to read from defunct connection IPv4Address(('3a6c4d5b.databases.neo4j.io', 7687)) (ResolvedIPv4Address(('34.121.155.65', 7687)))
Transaction failed and will be retried in 0.8010093927351241s (Failed to read from defunct connection IPv4Address(('3a6c4d5b.databases.neo4j.io', 7687)) (ResolvedIPv4Address(('34.121.155.65', 7687))))
Failed to read from defunct connection ResolvedIPv4Address(('34.121.155.65', 7687)) (ResolvedIPv4Address(('34.121.155.65', 7687)))
Unable to retrieve routing information
Transaction failed and will be retried in 1.8477170272253496s (Unable to retrieve routing information)
Unable to retrieve routing information
Transaction failed and will be retried in 3.337252686007687s (Unable to retrieve routing information)
Unable to retrieve routing information
Transaction failed and will be retried in 9.418069595505314s (Unable to retrieve routing information)
Unable to retrieve routing information
Transaction failed and will be retried in 14.70593468778483s (Unable

In [ ]:
import uuid
from blar_graph.graph_construction.core.graph_builder import GraphConstructor
from blar_graph.db_managers import Neo4jManager
from blar_graph.agents_tools.agents_examples.metamate import get_metamate_agent
import subprocess
import hashlib


graph_manager = downloaded_repos[repo]
agent = get_metamate_agent(graph_manager)
res = list(agent.stream({"input": f"Proble: {row['problem_statement']}\nHints: {row['hints_text']}"}))

NameError: name 'downloaded_repos' is not defined

## SWE-Agent + SWE + Blar agent evaluation

In [ ]:
import re


def extract_modified_files_and_functions(diff):
    modified_files = {}
    modified_lines = {}

    # Extract modified files
    file_regex = re.compile(r"^diff --git a/(.*?) b/\1", re.MULTILINE)
    files = file_regex.findall(diff)

    # Extract changed lines
    for line in diff.split("\n"):
        if line.startswith("+") or line.startswith("-"):
            line_info = line.split("@@")[-1].split(",")[0].split("+")[-1]
            file_name = line.split(" a/")[-1].split(" ")[0]
            if file_name not in modified_lines:
                modified_lines[file_name] = []
            line_number = line_info.split(" ")[0]
            if line_number.isdigit():
                modified_lines[file_name].append(line_number)

    for file_path in files:
        modified_files[file_path] = modified_lines.get(file_path, [])

    return modified_files


# Example usage:
diff = """diff --git a/src/sqlfluff/rules/L031.py b/src/sqlfluff/rules/L031.py
--- a/src/sqlfluff/rules/L031.py
+++ b/src/sqlfluff/rules/L031.py
@@ -1,5 +1,5 @@
 # -*- coding: utf-8 -*-
-from sqlfluff.core.rules.std_f001 import BaseCrawler
+from sqlfluff.core.rules.base import BaseCrawler
 from sqlfluff.core.rules.doc_decorators import document_fix_compatible
 from sqlfluff.core.rules.std_f001 import Rule

@@ -11,6 +11,6 @@ class Rule_L031(BaseCrawler):
         return [
             "select [1:distinct] star"
-            'from [1:tbl] as tbl_1'
+            'from [1:tbl] tbl_1'
         ]
"""

modified_files = extract_modified_files_and_functions(diff)

print(modified_files)

{'src/sqlfluff/rules/L031.py': []}


In [ ]:
import re
from collections import defaultdict


def extract_modified_files_and_functions(diff):
    modified_files = []
    modified_functions = {}
    changed_lines_dict = defaultdict(list)

    # Extract modified files
    file_regex = re.compile(r"^diff --git a/(.*?) b/\1", re.MULTILINE)
    files = file_regex.findall(diff)

    # Extract modified functions
    function_regex = re.compile(r"^@@.*?@@\s*def\s+(.*?)\(", re.MULTILINE)
    functions = function_regex.findall(diff)

    lines_changes_regex = re.compile(r"\+\+\+ b\/(.*?)\n@@ -(\d+),\d+ \+(\d+),\d+ @@", re.MULTILINE)
    lines_changes = lines_changes_regex.findall(diff)

    # Extract changed lines
    for line in lines_changes:
        changed_lines_dict[line[0]].append((int(line[1]), int(line[2])))

    for file_path in files:
        modified_files.append(file_path)
        modified_functions[file_path] = []

    for function_name in functions:
        for file_path in modified_files:
            modified_functions[file_path].append(function_name)

    return modified_files, modified_functions, dict(changed_lines_dict)


# Example usage
gold = {}
for data in dataset["dev"]:
    diff = data["patch"]
    modified_files, modified_functions, changed_lines = extract_modified_files_and_functions(diff)

    gold[data["instance_id"]] = {}
    gold[data["instance_id"]]["modified_files"] = modified_files
    gold[data["instance_id"]]["modified_functions"] = modified_functions
    gold[data["instance_id"]]["modified_lines"] = changed_lines
    # print(changed_lines)

    # print("Modified Files:")
    # print(modified_files)
    # print("\nModified Functions:")
    # print(modified_functions)
    # print("=" *80)

In [ ]:
import jsonlines


def process_jsonl_file(jsonl_file_path):
    results = {}
    with jsonlines.open(jsonl_file_path) as reader:
        for line in reader:
            diff = line["model_patch"]
            instance_id = line["instance_id"]
            if diff is None:
                continue

            modified_files, modified_functions, modified_lines = extract_modified_files_and_functions(diff)

            results[instance_id] = {}
            results[instance_id]["modified_files"] = modified_files
            results[instance_id]["modified_lines"] = modified_lines
            results[instance_id]["modified_functions"] = modified_functions
            # print("Modified Files:")
            # print(modified_files)
            # print("\nModified Functions:")
            # print(modified_functions)
            # print("\nModified Lines:")
            # print(modified_lines)
            # print("=" *80)
    return results


base = process_jsonl_file("base.jsonl")
blar1 = process_jsonl_file("blar1.jsonl")
blar2 = process_jsonl_file("blar2.jsonl")

In [ ]:
exclude_words = ["reproduce", "test", "issue"]


def compare_modified_files(your_results, gold_results):
    correct_changes = 0

    total = 0
    for instance_id, your_result in your_results.items():
        gold_result = gold_results.get(instance_id)
        if gold_result:
            your_modified_files = [
                file
                for file in your_result["modified_files"]
                if not any(word in file.lower() for word in exclude_words)
            ]

            your_modified_files = set(your_modified_files)
            gold_modified_files = set(gold_result["modified_files"])
            print(instance_id)
            print(your_modified_files)
            print(gold_modified_files)
            print("=" * 80)
            if gold_modified_files == your_modified_files:
                correct_changes += 1

            if your_modified_files:
                total += 1
    percent = correct_changes / total * 100
    formated_percent = "{:.2f}".format(percent)
    return formated_percent + "%", correct_changes, total


print(compare_modified_files(blar2, gold))
print(compare_modified_files(blar1, gold))
print(compare_modified_files(base, gold))

sqlfluff__sqlfluff-1625
{'src/sqlfluff/rules/L031.py'}
{'src/sqlfluff/rules/L031.py'}
sqlfluff__sqlfluff-2419
{'src/sqlfluff/rules/L060.py'}
{'src/sqlfluff/rules/L060.py'}
sqlfluff__sqlfluff-1733
{'src/sqlfluff/rules/L036.py', 'example.sql'}
{'src/sqlfluff/rules/L039.py'}
sqlfluff__sqlfluff-1517
set()
{'src/sqlfluff/core/parser/helpers.py'}
marshmallow-code__marshmallow-1359
{'src/marshmallow/fields.py'}
{'src/marshmallow/fields.py'}
pvlib__pvlib-python-1707
{'pvlib/iam.py'}
{'pvlib/iam.py'}
pvlib__pvlib-python-1606
{'pvlib/tools.py'}
{'pvlib/tools.py'}
sqlfluff__sqlfluff-1763
set()
{'src/sqlfluff/core/linter/linted_file.py'}
marshmallow-code__marshmallow-1343
{'src/marshmallow/schema.py'}
{'src/marshmallow/schema.py'}
pvlib__pvlib-python-1854
{'pvlib/pvsystem.py'}
{'pvlib/pvsystem.py'}
pylint-dev__astroid-1978
{'astroid/raw_building.py'}
{'astroid/raw_building.py'}
pvlib__pvlib-python-1072
{'pvlib/temperature.py'}
{'pvlib/temperature.py'}
pylint-dev__astroid-1333
{'root_dir/src/__init

In [ ]:
exclude_words = ["reproduce", "test", "issue"]


def compare_modified_lines(your_results, gold_results):
    correct_changes = 0

    total = 0
    for instance_id, your_result in your_results.items():
        gold_result = gold_results.get(instance_id)
        if gold_result:
            your_modified_files = [
                file
                for file in your_result["modified_files"]
                if not any(word in file.lower() for word in exclude_words)
            ]

            your_modified_files = set(your_modified_files)
            gold_modified_files = set(gold_result["modified_files"])

            for file in your_modified_files:
                your_modified_lines = your_result["modified_lines"].get(file, [])
                gold_modified_lines = gold_result["modified_lines"].get(file, [])
                if not gold_modified_lines or not your_modified_lines:
                    continue
                for line in your_modified_lines:
                    if any([abs(line[0] - gold_line[0]) < 10 for gold_line in gold_modified_lines]):
                        correct_changes += 1
                total += 1

    percent = correct_changes / total * 100
    formated_percent = "{:.2f}".format(percent)
    return formated_percent + "%", correct_changes, total


print(compare_modified_lines(blar2, gold))
print(compare_modified_lines(blar1, gold))
print(compare_modified_lines(base, gold))

('41.18%', 7, 17)
('43.75%', 7, 16)
('41.18%', 7, 17)


In [ ]:
blar2

{'sqlfluff__sqlfluff-1625': {'modified_files': ['src/sqlfluff/rules/L031.py'],
  'modified_functions': {'src/sqlfluff/rules/L031.py': []}},
 'sqlfluff__sqlfluff-2419': {'modified_files': ['src/sqlfluff/rules/L060.py'],
  'modified_functions': {'src/sqlfluff/rules/L060.py': []}},
 'sqlfluff__sqlfluff-1733': {'modified_files': ['example.sql',
   'src/sqlfluff/rules/L036.py'],
  'modified_functions': {'example.sql': [], 'src/sqlfluff/rules/L036.py': []}},
 'sqlfluff__sqlfluff-1517': {'modified_files': ['reproduce_bug.py'],
  'modified_functions': {'reproduce_bug.py': []}},
 'marshmallow-code__marshmallow-1359': {'modified_files': ['src/marshmallow/fields.py'],
  'modified_functions': {'src/marshmallow/fields.py': []}},
 'pvlib__pvlib-python-1707': {'modified_files': ['pvlib/iam.py',
   'reproduce_bug.py'],
  'modified_functions': {'pvlib/iam.py': ['physical'],
   'reproduce_bug.py': ['physical']}},
 'pvlib__pvlib-python-1606': {'modified_files': ['pvlib/tools.py',
   'reproduce_bug.py'],


In [ ]:
import re


def extract_modified_files_and_functions(diff):
    modified_files = {}
    modified_functions = {}
    changed_lines = {}

    # Extract modified files
    file_regex = re.compile(r"^diff --git a/(.*?) b/\1", re.MULTILINE)
    files = file_regex.findall(diff)

    # Extract modified functions
    function_regex = re.compile(r"^@@.*?@@\s*def\s+(.*?)\(", re.MULTILINE)
    functions = function_regex.findall(diff)

    # Extract changed lines
    for line in diff.split("\n"):
        if line.startswith("+") or line.startswith("-"):
            line_info = line.split("@@")[-1].split(",")[0].split("+")[-1]
            file_name = line.split(" a/")[-1].split(" ")[0]
            if file_name not in changed_lines:
                changed_lines[file_name] = []
            line_number = line_info.split(" ")[0]
            if line_number.isdigit():
                changed_lines[file_name].append((line_number, line_info.split(" ")[1]))

    for file_path in files:
        modified_files[file_path] = changed_lines.get(file_path, [])
        modified_functions[file_path] = functions

    return modified_files, modified_functions


# Example usage:
diff = """
diff --git a/astroid/brain/brain_builtin_inference.py b/astroid/brain/brain_builtin_inference.py
index af1ddf4d..f5dfa822 100644
--- a/astroid/brain/brain_builtin_inference.py
+++ b/astroid/brain/brain_builtin_inference.py
@@ -51,6 +51,20 @@ class whatever(object):
         return 0
     def find(self, sub, start=None, end=None):
         return 0
+    def capitalize(self):
+        return {rvalue}
+    def title(self):
+        return {rvalue}
+    def lower(self):
+        return {rvalue}
+    def upper(self):
+        return {rvalue}
+    def swapcase(self):
+        return {rvalue}
+    def index(self, sub, start=None, end=None):
+        return 0
+    def find(self, sub, start=None, end=None):
+        return 0
     def count(self, sub, start=None, end=None):
         return 0
     def strip(self, chars=None):
@@ -953,9 +967,13 @@ def _infer_str_format_call(
     keyword_values: dict[str, str] = {k: v.value for k, v in inferred_keyword.items()}
 
     try:
+        # Handle None values in pos_values and keyword_values before formatting
+        pos_values = ["" if v is None else v for v in pos_values]
+        keyword_values = {k: ("" if v is None else v) for k, v in keyword_values.items()}
         formatted_string = format_template.format(*pos_values, **keyword_values)
-    except (IndexError, KeyError):
-        # If there is an IndexError there are too few arguments to interpolate
+    except (IndexError, KeyError, ValueError):
+        # If there is an IndexError or KeyError, there are too few arguments to interpolate
+        # ValueError is raised when a format string is invalid
         return iter([util.Uninferable])
 
     return iter([nodes.const_factory(formatted_string)])
"""

modified_files, modified_functions = extract_modified_files_and_functions(diff)

print("Modified Files:")
print(modified_files)
print("\nModified Functions:")
print(modified_functions)

Modified Files:
{'astroid/brain/brain_builtin_inference.py': []}

Modified Functions:
{'astroid/brain/brain_builtin_inference.py': ['_infer_str_format_call']}
